# Data Cleaning For Business Data and Merging with Review

## Input
1. business_train.json
2. review_brunch.csv

## Output
1. 628_brunch_lv.csv

In [2]:
import json
import pandas as pd
import numpy as np
from numpy import *  # for what???
from operator import or_
import datetime  # For hours' format
import re   # For categories' matching
import os  # For working directory

In [6]:
os.getcwd()
os.chdir('../data')

In [7]:
df_review_brunch = pd.read_csv('review_brunch.csv', lineterminator='\n', index_col=0)

In [8]:
df_review_brunch.iloc[:3,:]

,business_id,date,stars,processed_text
10,40775.0,2017-12-15 23:27:08,1.0,walk around friday afternoon sit table bar wal...
19,64416.0,2016-07-25 03:57:19,5.0,pick meat planet chef make mexican style dish ...
35,145384.0,2018-05-28 20:56:05,5.0,party order fish taco pork belly banh mi corn ...


In [9]:
df_review_brunch.shape

(521672, 4)

In [10]:
bus_train = []
for line in open ('business_train.json','r'):
    bus_train.append(json.loads(line))

In [11]:
df_bus_train = pd.DataFrame.from_dict(bus_train)

In [12]:
cols = df_bus_train.columns.tolist()

In [13]:
newcols = [cols[1],cols[8],cols[3],cols[10],cols[9],cols[6],cols[7],cols[5],cols[0],cols[2],cols[4]]

In [14]:
df_bus_1 = df_bus_train[newcols]
attributes = df_bus_train['attributes']
hours = df_bus_train['hours']

In [15]:
all_attributes = set(attributes[0].keys())
for i in range(1,len(attributes)): 
    if pd.isna(attributes[i])==False:
        all_attributes = all_attributes.union(set(attributes[i].keys()))
len(all_attributes)

39

In [16]:
all_attributes = list(all_attributes)

In [17]:
all_hrs = []
for i in range(len(hours)): 
    if pd.isna(hours[i])==False:
        all_hrs = list(set(all_hrs + list(hours[i].keys())))


In [18]:
for i in range(len(all_attributes)):
    alist = []
    for j in range(len(attributes)):
            if pd.isna(attributes[j])==False:
                if all_attributes[i] in set(attributes[j].keys()):
                    alist.append(attributes[j][all_attributes[i]])
                else: alist.append('NA')
            else: alist.append('NA')
    df_bus_1[all_attributes[i]]=alist

C:\Users\25982\AppData\Local\conda\conda\envs\stat 628\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
for i in range(len(all_hrs)):
    blist = []
    for j in range(len(hours)):
            if pd.isna(hours[j])==False:
                if all_hrs[i] in set(hours[j].keys()):
                    blist.append(hours[j][all_hrs[i]])
                else: blist.append('NA')
            else: blist.append('NA')
    df_bus_1[all_hrs[i]]=blist

C:\Users\25982\AppData\Local\conda\conda\envs\stat 628\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
df_bus_1[['Monday_open','Monday_close']]=df_bus_1.pop('Monday').str.extract('(.*)-(.*)',  expand=True)
df_bus_1[['Tuesday_open','Tuesday_close']]=df_bus_1.pop('Tuesday').str.extract('(.*)-(.*)',  expand=True)
df_bus_1[['Wednesday_open','Wednesday_close']]=df_bus_1.pop('Wednesday').str.extract('(.*)-(Sunday.*)',  expand=True)
df_bus_1[['Thursday_open','Thursday_close']]=df_bus_1.pop('Thursday').str.extract('(.*)-(.*)',  expand=True)
df_bus_1[['Friday_open','Friday_close']]=df_bus_1.pop('Friday').str.extract('(.*)-(.*)',  expand=True)
df_bus_1[['Saturday_open','Saturday_close']]=df_bus_1.pop('Saturday').str.extract('(.*)-(.*)',  expand=True)
df_bus_1[['Sunday_open','Sunday_close']]=df_bus_1.pop('Sunday').str.extract('(.*)-(.*)',  expand=True)

C:\Users\25982\AppData\Local\conda\conda\envs\stat 628\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [21]:
week_day = ["Monday", "Tuesday" ,"Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
for temp_day in week_day:
    time_open = df_bus_1[temp_day+'_open'].replace(np.nan, '', regex=True)
    time_open_datetime = (pd.Series ([datetime.datetime.strptime(item, "%H:%M") if item != '' else None
                                     for item in time_open ]))
    time_close = df_bus_1[temp_day+'_close'].replace(np.nan, '', regex=True)
    time_close_datetime = (pd.Series ([datetime.datetime.strptime(item, "%H:%M") if item != '' else None
                                     for item in time_close ]))
    time_diff_1=(time_close_datetime-time_open_datetime)/np.timedelta64(1, 'h')
    time_diff_1[time_diff_1 <= 0]=time_diff_1[time_diff_1 <= 0]+24
    time_diff_1[time_diff_1.isnull() & df_bus_1[['hours']].notnull().iloc[:,0]] = 0
    df_bus_1[temp_day+"_hours"] = time_diff_1

C:\Users\25982\AppData\Local\conda\conda\envs\stat 628\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [22]:
for temp_day in week_day:
        df_bus_1=df_bus_1.drop(columns=[temp_day+'_open',temp_day+'_close'])

In [23]:
df_bus_brunch=df_bus_1[df_bus_1['categories'].str.contains('Breakfast & Brunch', na=False)]

In [24]:
df_bus_brunch.shape

(4322, 57)

In [30]:
sum( df_bus_brunch['city'].str.match('Las Vegas', na=False))

448

In [26]:
df_merge = df_review_brunch.merge(df_bus_brunch, how = 'left', on = 'business_id')

In [27]:
df_merge_brunch = df_merge[df_merge['categories'].str.contains('Breakfast & Brunch', na=False)]
df_merge_brunch.shape

(521672, 60)

In [31]:
df_merge_brunch_lv = df_merge_brunch[df_merge_brunch['city'].str.match('Las Vegas', na=False)]
df_merge_brunch_lv.shape

(169529, 60)

In [33]:
df_merge_brunch_lv.to_csv('628_brunch_lv.csv')